<a href="https://colab.research.google.com/github/KoreTane/KoreTane/blob/main/ProfitForecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# Ler a aba "Receita"
df_receita = pd.read_excel('fluxo_caixa_cbs.xlsx', sheet_name='Receita')

# Ler a aba "Despesa"
df_despesa = pd.read_excel('fluxo_caixa_cbs.xlsx', sheet_name='Despesa')

In [3]:
df_despesa

,Data,Descrição,Valor,Tipo
0,2023-01-03,Sabesp,-59.46,Retirada pessoal
1,2023-01-05,EDP,-226.15,Despesas
2,2023-01-05,Sabesp,-119.15,Despesas
3,2023-01-09,EDP,-97.71,Retirada pessoal
4,2023-01-12,Internet,-140.12,Despesas
...,...,...,...,...
489,2024-04-23,Instalação bebedouro,-209.85,Reforma/Manutenção
490,2024-04-24,Suprimentos,-21.29,Despesas
491,2024-04-24,Suprimentos,-17.44,Despesas
492,2024-04-24,Tomada bebedouro,-26.90,Reforma/Manutenção


In [4]:
df_receita

,Data,Descrição,Valor,Tipo
0,2023-01-11,#CLIENTE#23017,244.02,Debito
1,2023-01-12,#CLIENTE#23018,199.00,Dinheiro
2,2023-01-13,#CLIENTE#23002,237.12,Crédito
3,2023-01-13,#CLIENTE#23003,237.12,Crédito
4,2023-01-14,#CLIENTE#23007,250.00,Dinheiro
...,...,...,...,...
1306,2024-04-15,#CLIENTE#24024,230.00,Dinheiro
1307,2024-04-17,#CLIENTE#24024,164.07,Crédito
1308,2024-04-23,#CLIENTE#24023,316.89,Crédito
1309,2024-04-23,#CLIENTE#24023,358.34,Crédito


In [5]:
# Agrupando por 'Tipo' e somando os 'Valor'
result_receita = df_receita.groupby('Tipo')['Valor'].sum().reset_index()
result_despesa = df_despesa.groupby('Tipo')['Valor'].sum().reset_index()

In [6]:
result_despesa
result_receita

,Tipo,Valor
0,Aporte pessoal,522.13
1,Boleto,38017.52
2,Crédito,301170.75
3,Debito,6443.54
4,Dinheiro,105575.28
5,Espetáculo,28161.19
6,Investimento,81.62


In [7]:
result_despesa

,Tipo,Valor
0,Despesas,-161024.25
1,Débito,-4076.38
2,Empréstimo,-54580.25
3,Impostos,-27555.00
4,Pró-labore,-22673.46
5,Reforma/Manutenção,-9738.55
6,Retirada pessoal,-44644.30
7,Salários,-176072.41
8,Tarifas Bancárias,-1312.64


In [8]:
print(result_despesa)
print(result_receita)

                 Tipo      Valor
0            Despesas -161024.25
1              Débito   -4076.38
2          Empréstimo  -54580.25
3            Impostos  -27555.00
4          Pró-labore  -22673.46
5  Reforma/Manutenção   -9738.55
6    Retirada pessoal  -44644.30
7            Salários -176072.41
8   Tarifas Bancárias   -1312.64
             Tipo      Valor
0  Aporte pessoal     522.13
1          Boleto   38017.52
2         Crédito  301170.75
3          Debito    6443.54
4        Dinheiro  105575.28
5      Espetáculo   28161.19
6    Investimento      81.62


In [9]:
despesa_agrupada = result_despesa.groupby('Tipo')['Valor'].sum().reset_index()

# Renomear a coluna para maior clareza
despesa_agrupada.columns = ['Descrição', 'Total']

# Exibir a matriz agrupada
print("Matriz de Despesa Agrupada:")
print(despesa_agrupada)

Matriz de Despesa Agrupada:
            Descrição      Total
0            Despesas -161024.25
1              Débito   -4076.38
2          Empréstimo  -54580.25
3            Impostos  -27555.00
4          Pró-labore  -22673.46
5  Reforma/Manutenção   -9738.55
6    Retirada pessoal  -44644.30
7            Salários -176072.41
8   Tarifas Bancárias   -1312.64


In [10]:
receita_agrupada = result_receita.groupby('Tipo')['Valor'].sum().reset_index()

# Renomear a coluna para maior clareza
receita_agrupada.columns = ['Tipo', 'ValorTotal']

# Exibir a matriz agrupada
print("Matriz de Receita Agrupada:")
print(receita_agrupada)

Matriz de Receita Agrupada:
             Tipo  ValorTotal
0  Aporte pessoal      522.13
1          Boleto    38017.52
2         Crédito   301170.75
3          Debito     6443.54
4        Dinheiro   105575.28
5      Espetáculo    28161.19
6    Investimento       81.62


In [11]:
# Agrupar receitas e despesas por Data
receita_agrupada = df_receita.groupby('Data')['Valor'].sum().reset_index()
receita_agrupada.rename(columns={'Valor': 'Receita'}, inplace=True)

despesa_agrupada = df_despesa.groupby('Data')['Valor'].sum().reset_index()
despesa_agrupada.rename(columns={'Valor': 'Despesa'}, inplace=True)

# Tornar as despesas negativas
despesa_agrupada['Despesa'] = -despesa_agrupada['Despesa']  # Tornar as despesas como valores negativos

# Unir os dois DataFrames
lucro_df = pd.merge(receita_agrupada, despesa_agrupada, on='Data', how='outer').fillna(0)

# Calcular o lucro corretamente usando as colunas do DataFrame lucro_df
lucro_df['Lucro'] = lucro_df['Receita'] + lucro_df['Despesa']  # Lucro = Receita - Despesa (que é negativa)

# Exibir o DataFrame de lucro
print(lucro_df)


          Data  Receita  Despesa    Lucro
0   2023-01-11   244.02     0.00   244.02
1   2023-01-12   199.00  -140.12    58.88
2   2023-01-13   474.24     0.00   474.24
3   2023-01-14   250.00 -7885.62 -7635.62
4   2023-01-16   374.95  -131.98   242.97
..         ...      ...      ...      ...
310 2024-04-11     0.00  -197.50  -197.50
311 2024-04-12     0.00  -180.34  -180.34
312 2024-04-18     0.00  -913.10  -913.10
313 2024-04-20     0.00 -4800.11 -4800.11
314 2024-04-30     0.00  -427.98  -427.98

[315 rows x 4 columns]


In [14]:
# Converter a coluna 'Data' para datetime
df_receita['Data'] = pd.to_datetime(df_receita['Data'])
df_despesa['Data'] = pd.to_datetime(df_despesa['Data'])

# Agrupar receitas e despesas por mês
receita_agrupada = df_receita.resample('M', on='Data')['Valor'].sum().reset_index()
receita_agrupada.rename(columns={'Valor': 'Receita'}, inplace=True)

despesa_agrupada = df_despesa.resample('M', on='Data')['Valor'].sum().reset_index()
despesa_agrupada.rename(columns={'Valor': 'Despesa'}, inplace=True)

# Unir os dois DataFrames
lucro_df = pd.merge(receita_agrupada, despesa_agrupada, on='Data', how='outer').fillna(0)

# Calcular o lucro
lucro_df['Lucro'] = lucro_df['Receita'] + lucro_df['Despesa']  # Lucro = Receita - Despesa (que é negativa)

# Criar subgráficos
fig = make_subplots(rows=1, cols=1,
                    subplot_titles=('Comparação Mensal entre Receitas, Despesas e Lucro',))

# Gráfico 1: Comparação Mensal de Receitas, Despesas e Lucro Total
fig.add_trace(go.Bar(
    x=lucro_df['Data'],
    y=lucro_df['Receita'],
    name='Receitas',
    marker_color='#007bff'
), row=1, col=1)

fig.add_trace(go.Bar(
    x=lucro_df['Data'],
    y=lucro_df['Despesa'],
    name='Despesas',
    marker_color='#ff7b00'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=lucro_df['Data'],
    y=lucro_df['Lucro'],
    mode='lines+markers+text',  # Adiciona texto aos marcadores
    name='Lucro',
    marker_color='#00cc00',
    line_color='#00cc00',
    text=lucro_df['Lucro'].round(2),  # Rótulos com os valores do lucro
    textposition='top center'  # Posição do texto acima dos pontos
), row=1, col=1)

# Personalizar o layout do gráfico
fig.update_layout(
    title='Análise Financeira Mensal: Receitas, Despesas e Lucro',
    xaxis_title='Mês',
    yaxis_title='Valor (R$)',
    font_family='Arial',
    font_color='#333333',
    title_font_size=20,
    xaxis_tickfont_size=12,
    yaxis_tickfont_size=12,
    bargap=0.1,
    plot_bgcolor='#f5f5f5',
    paper_bgcolor='#f5f5f5'
)

# Exibir o gráfico
fig.show()


In [13]:
df_despesa[df_despesa['Tipo'] == 'Despesas'].groupby('Tipo')['Valor'].sum().reset_index()

,Tipo,Valor
0,Despesas,-161024.25
